In [81]:
import requests
import pandas as pd
import time


class SimFinAPIs:
    """
    A simple API wrapper for SimFin v3, handling share prices, income statements, and balance sheets.
    Replace 'b7f5ad1b-6cd9-4f19-983b-cfddaad8df9c' with your actual API key.
    """
    def __init__(self, api_key="b7f5ad1b-6cd9-4f19-983b-cfddaad8df9c"):
        self.api_key = api_key
        self.base_url = "https://backend.simfin.com/api/v3/"
        self.headers = {
            "Authorization": f"{self.api_key}",
            "accept": "application/json"
        }
        self.rate_limit = 0.5  # Respect SimFin's API rate limit (2 requests/sec)

    def _respect_rate_limit(self):
        """Ensures requests comply with SimFin's rate limits."""
        time.sleep(self.rate_limit)

    def _make_request(self, url, params=None):
        """Handles API requests with rate limiting and error handling."""
        self._respect_rate_limit()
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            return response.json()  # Return raw JSON
        except requests.exceptions.HTTPError as e:
            print(f"HTTP Error {response.status_code}: {response.text}")
            return []
        except Exception as e:
            print(f"Request error: {e}")
            return []
    
    def get_share_prices(self, ticker, start_date, end_date):
        """Fetches daily share prices for a ticker using the v3 API."""
        url = f"{self.base_url}companies/prices/compact"
        params = {
            "ticker": ticker.upper(),
            "start": start_date,
            "end": end_date
        }
        data = self._make_request(url, params)

        if not data or not isinstance(data, list) or len(data) == 0:
            print(f"No price data for {ticker} between {start_date} and {end_date}")
            return pd.DataFrame(columns=['date', 'ticker', 'close'])

        columns = data[0].get("columns", [])
        try:
            date_idx = columns.index("Date")
            close_idx = columns.index("Last Closing Price")
        except ValueError:
            print("Error: Expected columns not found in API response.")
            return pd.DataFrame(columns=['date', 'ticker', 'close'])

        processed_data = [
            {"date": pd.to_datetime(row[date_idx]), "ticker": ticker.upper(), "close": row[close_idx]}
            for row in data[0].get("data", []) if len(row) > close_idx
        ]

        df = pd.DataFrame(processed_data).dropna()
        return df.sort_values(by="date", ascending=True)
    
    def get_income_statement(self, ticker, start_date, end_date):
        """Fetches the income statement data for a ticker."""
        url = f"{self.base_url}companies/statements/compact"
        params = {
            "ticker": ticker.upper(),
            "statements": "PL",
            "period": "Q1,Q2,Q3,Q4",  # Quarterly statements
            "start": start_date,
            "end": end_date
        }
        data = self._make_request(url, params)

        if not data or not isinstance(data, list) or len(data) == 0:
            print(f"No income data for {ticker} between {start_date} and {end_date}")
            return pd.DataFrame(columns=['ticker', 'date', 'fiscal_period', 'fiscal_year', 'revenue', 'net_income'])

        statements = data[0].get("statements", [])
        pl_statement = statements[0] if statements else {}

        columns = pl_statement.get("columns", [])
        try:
            fiscal_period_idx = columns.index("Fiscal Period")
            fiscal_year_idx = columns.index("Fiscal Year")
            report_date_idx = columns.index("Report Date")
            revenue_idx = columns.index("Revenue")
            net_income_idx = columns.index("Net Income")
        except ValueError:
            print("Error: Expected columns not found in API response.")
            return pd.DataFrame(columns=['ticker', 'date', 'fiscal_period', 'fiscal_year', 'revenue', 'net_income'])

        processed_data = [
            {
                "ticker": ticker.upper(),
                "date": pd.to_datetime(row[report_date_idx], errors='coerce'),
                "fiscal_period": row[fiscal_period_idx],
                "fiscal_year": row[fiscal_year_idx],
                "revenue": pd.to_numeric(row[revenue_idx], errors='coerce'),
                "net_income": pd.to_numeric(row[net_income_idx], errors='coerce')
            }
            for row in pl_statement.get("data", []) if len(row) > max(report_date_idx, revenue_idx, net_income_idx)
        ]

        df = pd.DataFrame(processed_data).dropna()
        return df.sort_values(by="date", ascending=True)
    
    def get_balance_sheet(self, ticker, start_date, end_date):
        """Fetches balance sheet data for a ticker."""
        url = f"{self.base_url}companies/statements/compact"
        params = {
            "ticker": ticker.upper(),
            "statements": "BS",
            "start": start_date,
            "end": end_date
        }
        data = self._make_request(url, params)

        if not data or not isinstance(data, list) or len(data) == 0:
            print(f"No balance sheet data for {ticker} between {start_date} and {end_date}")
            return pd.DataFrame(columns=['ticker', 'date', 'totalLiabilities', 'totalEquity', 'share_capital'])

        statements = data[0].get("statements", [])
        bs_statement = statements[0] if statements else {}

        columns = bs_statement.get("columns", [])
        try:
            date_idx = columns.index("Report Date")
            liabilities_idx = columns.index("Total Liabilities")
            equity_idx = columns.index("Total Equity")
            share_capital_idx = columns.index("Share Capital & Additional Paid-In Capital")
        except ValueError:
            print("Error: Expected columns not found in API response.")
            return pd.DataFrame(columns=['ticker', 'date', 'totalLiabilities', 'totalEquity', 'share_capital'])

        processed_data = [
            {
                "date": pd.to_datetime(row[date_idx], errors='coerce'),
                "ticker": ticker.upper(),
                "totalLiabilities": pd.to_numeric(row[liabilities_idx], errors='coerce'),
                "totalEquity": pd.to_numeric(row[equity_idx], errors='coerce'),
                "share_capital": pd.to_numeric(row[share_capital_idx], errors='coerce')
            }
            for row in bs_statement.get("data", []) if len(row) > max(date_idx, liabilities_idx, equity_idx, share_capital_idx)
        ]

        df = pd.DataFrame(processed_data).dropna()
        return df.sort_values(by="date", ascending=True)
    



In [82]:
# 🚀 **Test API Calls**
if __name__ == "__main__":
    api = SimFinAPI("b7f5ad1b-6cd9-4f19-983b-cfddaad8df9c")

    # **Test get_share_prices**
    print("\nFetching Apple (AAPL) Share Prices from past year...")
    share_prices_df = api.get_share_prices("AAPL", "2023-12-30", "2025-03-06")
    print(share_prices_df.head(10) if not share_prices_df.empty else "No share price data retrieved.")

    # **Test get_income_statement**
    print("\nFetching Apple (AAPL) Income Statement from past year...")
    income_df = api.get_income_statement("AAPL", "2023-12-30", "2025-03-06")
    print(income_df.head(10) if not income_df.empty else "No income statement data retrieved.")

    # **Test get_balance_sheet**
    print("\nFetching Apple (AAPL) Balance Sheet from past year...")
    balance_sheet_df = api.get_balance_sheet("AAPL", "2023-12-30", "2025-03-06")
    print(balance_sheet_df.head(10) if not balance_sheet_df.empty else "No balance sheet data retrieved.")



Fetching Apple (AAPL) Share Prices from past year...
        date ticker   close
0 2024-01-02   AAPL  185.64
1 2024-01-03   AAPL  184.25
2 2024-01-04   AAPL  181.91
3 2024-01-05   AAPL  181.18
4 2024-01-08   AAPL  185.56
5 2024-01-09   AAPL  185.14
6 2024-01-10   AAPL  186.19
7 2024-01-11   AAPL  185.59
8 2024-01-12   AAPL  185.92
9 2024-01-16   AAPL  183.63

Fetching Apple (AAPL) Income Statement from past year...
  ticker       date fiscal_period  fiscal_year       revenue   net_income
0   AAPL 2023-12-31            Q1         2024  119575000000  33916000000
2   AAPL 2024-03-31            Q2         2024   90753000000  23636000000
3   AAPL 2024-06-30            Q3         2024   85777000000  21448000000
4   AAPL 2024-09-30            Q4         2024   94930000000  14736000000
1   AAPL 2024-12-31            Q1         2025  124300000000  36330000000

Fetching Apple (AAPL) Balance Sheet from past year...
        date ticker  totalLiabilities  totalEquity  share_capital
0 2023-12-31   

In [78]:
display(merged_df)

,date,ticker,close,fiscal_period,fiscal_year,revenue,net_income,totalLiabilities,totalEquity,share_capital,earnings_per_share,p_e_ratio,sma_50,next_close
187,2024-09-30,AAPL,233.00,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1316.728284,222.2656,226.21
188,2024-10-01,AAPL,226.21,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1278.356675,222.3106,226.78
189,2024-10-02,AAPL,226.78,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1281.577856,222.3460,225.67
190,2024-10-03,AAPL,225.67,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1275.305030,222.4886,226.80
191,2024-10-04,AAPL,226.80,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1281.690879,222.6748,221.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,2025-02-28,AAPL,241.84,Q1,2025.0,1.243000e+11,3.633000e+10,2.773270e+11,6.675800e+10,8.476800e+10,0.428582,564.280020,240.2210,238.03
291,2025-03-03,AAPL,238.03,Q1,2025.0,1.243000e+11,3.633000e+10,2.773270e+11,6.675800e+10,8.476800e+10,0.428582,555.390230,239.9608,235.93
292,2025-03-04,AAPL,235.93,Q1,2025.0,1.243000e+11,3.633000e+10,2.773270e+11,6.675800e+10,8.476800e+10,0.428582,550.490345,239.6098,235.74
293,2025-03-05,AAPL,235.74,Q1,2025.0,1.243000e+11,3.633000e+10,2.773270e+11,6.675800e+10,8.476800e+10,0.428582,550.047022,239.3636,235.33


In [74]:


# Convert date columns to datetime format for proper merging
balance_sheet_df["date"] = pd.to_datetime(balance_sheet_df["date"])
income_df["date"] = pd.to_datetime(income_df["date"])
share_prices_df["date"] = pd.to_datetime(share_prices_df["date"])

# Merge datasets using a left join on ticker and date
merged_df = share_prices_df.merge(income_df, on=["ticker", "date"], how="left")
merged_df = merged_df.merge(balance_sheet_df, on=["ticker", "date"], how="left")

# Sort by ticker and date (most recent to oldest)
merged_df = merged_df.sort_values(by=["date"], ascending=[True])

# Forward-fill missing values for financial data
merged_df.fillna(method="ffill", inplace=True)

# Compute P/E ratio (Price-to-Earnings Ratio)
merged_df["earnings_per_share"] = merged_df["net_income"] / merged_df["share_capital"]
merged_df["p_e_ratio"] = merged_df["close"] / merged_df["earnings_per_share"]

# Compute 50-day Simple Moving Average (SMA)
merged_df["sma_50"] = merged_df.groupby("ticker")["close"].transform(lambda x: x.rolling(window=50, min_periods=1).mean())

# Add next day's close price as a target variable
merged_df["next_close"] = merged_df.groupby("ticker")["close"].shift(-1)

# Drop rows where critical features contain NaN values
merged_df = merged_df.dropna(subset=["close", "p_e_ratio", "sma_50"])

# Display the first few rows of the final dataset
display(merged_df)

,date,ticker,close,fiscal_period,fiscal_year,revenue,net_income,totalLiabilities,totalEquity,share_capital,earnings_per_share,p_e_ratio,sma_50,next_close
187,2024-09-30,AAPL,233.00,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1316.728284,222.2656,226.21
188,2024-10-01,AAPL,226.21,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1278.356675,222.3106,226.78
189,2024-10-02,AAPL,226.78,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1281.577856,222.3460,225.67
190,2024-10-03,AAPL,225.67,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1275.305030,222.4886,226.80
191,2024-10-04,AAPL,226.80,Q4,2024.0,9.493000e+10,1.473600e+10,3.080300e+11,5.695000e+10,8.327600e+10,0.176954,1281.690879,222.6748,221.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,2025-02-28,AAPL,241.84,Q1,2025.0,1.243000e+11,3.633000e+10,2.773270e+11,6.675800e+10,8.476800e+10,0.428582,564.280020,240.2210,238.03
291,2025-03-03,AAPL,238.03,Q1,2025.0,1.243000e+11,3.633000e+10,2.773270e+11,6.675800e+10,8.476800e+10,0.428582,555.390230,239.9608,235.93
292,2025-03-04,AAPL,235.93,Q1,2025.0,1.243000e+11,3.633000e+10,2.773270e+11,6.675800e+10,8.476800e+10,0.428582,550.490345,239.6098,235.74
293,2025-03-05,AAPL,235.74,Q1,2025.0,1.243000e+11,3.633000e+10,2.773270e+11,6.675800e+10,8.476800e+10,0.428582,550.047022,239.3636,235.33


In [75]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108 entries, 187 to 294
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                108 non-null    datetime64[ns]
 1   ticker              108 non-null    object        
 2   close               108 non-null    float64       
 3   fiscal_period       108 non-null    object        
 4   fiscal_year         108 non-null    float64       
 5   revenue             108 non-null    float64       
 6   net_income          108 non-null    float64       
 7   totalLiabilities    108 non-null    float64       
 8   totalEquity         108 non-null    float64       
 9   share_capital       108 non-null    float64       
 10  earnings_per_share  108 non-null    float64       
 11  p_e_ratio           108 non-null    float64       
 12  sma_50              108 non-null    float64       
 13  next_close          107 non-null    float64       
dt

In [40]:
print(merged_df['date'].min(), merged_df['date'].max())

2024-09-30 00:00:00 2025-03-06 00:00:00


In [25]:
share_prices_df.to_csv("share_prices_df", index=False)

In [26]:
income_df.to_csv("income_df", index=False)

In [27]:
balance_sheet_df.to_csv("balance_sheet_df", index=False)